In [24]:
import numpy as np
import pandas as pd

In [25]:
MC_compare_models_grid_test_numba = pd.read_csv('Data/evaluation/MC_compare_models_grid_test_numba.csv', index_col=0)
tmp = MC_compare_models_grid_test_numba.drop(['heston_params#','batch_size','absolute error'], axis=1)
speed_test = pd.DataFrame()


speed_test[['strike', 'N_T']] = MC_compare_models_grid_test_numba.loc[MC_compare_models_grid_test_numba['scheme'] =='simulate_heston_euler',['strike', 'N_T']]

speed_test['qe_time'] = tmp.loc[tmp['scheme'] =='simulate_heston_andersen_qe', 'time'].reset_index(drop=True)
speed_test['qe_relative_speed'] = tmp.loc[tmp['scheme'] =='simulate_heston_andersen_qe', 'time'].reset_index(drop=True)/tmp.loc[tmp['scheme'] =='simulate_heston_euler', 'time'].reset_index(drop=True)
speed_test['qe_error'] = tmp.loc[tmp['scheme'] =='simulate_heston_andersen_qe', 'MC_price'].reset_index(drop=True) - tmp.loc[tmp['scheme'] =='simulate_heston_andersen_qe', 'true'].reset_index(drop=True)

speed_test['tg_time'] = tmp.loc[tmp['scheme'] =='simulate_heston_andersen_tg', 'time'].reset_index(drop=True)
speed_test['tg_relative_speed'] = tmp.loc[tmp['scheme'] =='simulate_heston_andersen_tg', 'time'].reset_index(drop=True)/tmp.loc[tmp['scheme'] =='simulate_heston_euler', 'time'].reset_index(drop=True)
speed_test['tg_error'] = tmp.loc[tmp['scheme'] =='simulate_heston_andersen_tg', 'MC_price'].reset_index(drop=True) - tmp.loc[tmp['scheme'] =='simulate_heston_andersen_tg', 'true'].reset_index(drop=True)

speed_test['euler_time'] = tmp.loc[tmp['scheme'] =='simulate_heston_euler', 'time'].reset_index(drop=True)
speed_test['euler_error'] = tmp.loc[tmp['scheme'] =='simulate_heston_euler', 'MC_price'].reset_index(drop=True) - tmp.loc[tmp['scheme'] =='simulate_heston_euler', 'true'].reset_index(drop=True)

In [26]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=3, specs=[[{'type':'surface'}, {'type':'surface'}, {'type':'surface'}]], subplot_titles = ['Euler', 'Andersen Truncated Gaussian', 'Andersen Quadratic-Exponential'])

tmp_euler_error = speed_test.pivot(index='strike', columns='N_T', values='euler_error').abs()
fig.add_trace(go.Surface(z=tmp_euler_error.values, x=tmp_euler_error.columns, y=tmp_euler_error.index, showscale=False), row=1, col=1, )

tmp_tg_error = speed_test.pivot(index='strike', columns='N_T', values='tg_error').abs()
fig.add_trace(go.Surface(z=tmp_tg_error.values, x=tmp_tg_error.columns, y=tmp_tg_error.index, showscale=False), row=1, col=2)

tmp_qe_error = speed_test.pivot(index='strike', columns='N_T', values='qe_error').abs()
fig.add_trace(go.Surface(z=tmp_qe_error.values, x=tmp_qe_error.columns, y=tmp_qe_error.index, showscale=False), row=1, col=3)




max_error = max(tmp_euler_error.values.max(), tmp_qe_error.values.max(), tmp_tg_error.values.max())
min_error  =min(tmp_euler_error.values.min(), tmp_qe_error.values.min(), tmp_tg_error.values.min())

fig.update_scenes(xaxis_title_text='N_T',  
                  yaxis_title_text='Strike',  
                  zaxis_title_text='error',
                     zaxis = dict(nticks=4, range=[min_error*(1.02),max_error*(1.02)]))

fig.update_layout(title='Errors',
    height=700)
fig.show()

In [28]:
tmp_tg_error.T

strike,60.000000,64.210526,68.421053,72.631579,76.842105,81.052632,85.263158,89.473684,93.684211,97.894737,102.105263,106.315789,110.526316,114.736842,118.947368,123.157895,127.368421,131.578947,135.789474,140.000000
N_T,,,,,,,,,,,,,,,,,,,,
5,0.146171,0.149358,0.161058,0.165665,0.169970,0.162049,0.157130,0.136431,0.096254,0.032632,0.193475,0.456382,0.100612,0.035271,0.015526,0.009670,0.004570,0.002634,0.000909,0.000742
10,0.105394,0.113197,0.119233,0.122741,0.124338,0.126412,0.124743,0.113163,0.086073,0.043601,0.077350,0.267274,0.051112,0.016551,0.007394,0.003668,0.000396,0.000228,0.000133,0.000547
15,0.077773,0.079735,0.089972,0.093369,0.096190,0.096872,0.096309,0.094365,0.074794,0.039522,0.047902,0.189813,0.038184,0.010704,0.002605,0.001101,0.000402,0.000637,0.000612,0.000262
20,0.063043,0.069526,0.073090,0.078847,0.078271,0.080514,0.084799,0.077845,0.067780,0.036427,0.033257,0.143862,0.031746,0.008527,0.003737,0.001520,0.002021,0.000289,0.000344,0.000337
25,0.053525,0.054723,0.065992,0.063320,0.064344,0.071143,0.069608,0.058874,0.056815,0.038917,0.022573,0.116785,0.023325,0.008032,0.002701,0.000488,0.000788,0.000450,0.000991,0.001378
30,0.046838,0.045967,0.052394,0.056033,0.060107,0.061058,0.058022,0.054251,0.051252,0.029071,0.020552,0.104093,0.020998,0.005294,0.000864,0.000083,0.000355,0.000179,0.000363,0.000062
35,0.037842,0.040934,0.046835,0.045691,0.046876,0.050614,0.051429,0.049320,0.044118,0.032129,0.015996,0.089923,0.011727,0.005058,0.002053,0.001027,0.000627,0.000503,0.000137,0.000553
40,0.034704,0.038038,0.043046,0.043129,0.042796,0.045670,0.048317,0.047046,0.045332,0.029492,0.011226,0.079993,0.015541,0.002606,0.000239,0.000311,0.000132,0.001150,0.000153,0.000507
45,0.026539,0.032693,0.038725,0.039385,0.043770,0.040135,0.043762,0.043396,0.034187,0.027534,0.008951,0.071149,0.016724,0.005283,0.001420,0.000359,0.000598,0.001431,0.000974,0.000107


In [29]:
tmp_qe_error.T

strike,60.000000,64.210526,68.421053,72.631579,76.842105,81.052632,85.263158,89.473684,93.684211,97.894737,102.105263,106.315789,110.526316,114.736842,118.947368,123.157895,127.368421,131.578947,135.789474,140.000000
N_T,,,,,,,,,,,,,,,,,,,,
5,0.055782,0.046182,0.032635,0.027507,0.025585,0.017462,0.010099,0.000206,0.015499,0.062278,0.230900,0.440164,0.106322,0.038848,0.018181,0.010624,0.006452,0.004120,0.002131,0.001505
10,0.034919,0.027573,0.022406,0.022970,0.022484,0.019731,0.013619,0.007398,0.006574,0.031792,0.107819,0.231299,0.057870,0.016465,0.010986,0.005431,0.003172,0.002508,0.001134,0.000730
15,0.025454,0.018705,0.022258,0.019603,0.019313,0.016705,0.011986,0.005731,0.006164,0.025550,0.078104,0.150135,0.036557,0.014115,0.004847,0.004202,0.002488,0.001278,0.000753,0.000215
20,0.019216,0.018013,0.016406,0.016534,0.016007,0.012644,0.010916,0.004710,0.003312,0.017951,0.057030,0.109598,0.024999,0.011337,0.004341,0.004471,0.001646,0.000095,0.001285,0.000447
25,0.019020,0.013460,0.011006,0.012759,0.006802,0.009955,0.010151,0.003353,0.005285,0.013328,0.049660,0.088055,0.022001,0.011404,0.004855,0.002251,0.000585,0.000923,0.000139,0.000002
30,0.012764,0.011621,0.014832,0.012178,0.015395,0.007937,0.004016,0.000976,0.002170,0.013750,0.037877,0.077211,0.019637,0.004952,0.003371,0.001287,0.002013,0.001091,0.000068,0.000215
35,0.015166,0.013309,0.012423,0.011718,0.012892,0.010546,0.010198,0.005788,0.006628,0.011001,0.030837,0.067688,0.014755,0.003305,0.002210,0.002616,0.000222,0.000334,0.000139,0.000436
40,0.009163,0.014522,0.007685,0.008635,0.009566,0.005398,0.006367,0.002000,0.000692,0.004398,0.030034,0.056656,0.016058,0.004181,0.001765,0.000433,0.001678,0.000845,0.001760,0.000225
45,0.012560,0.009652,0.009613,0.004116,0.009909,0.008732,0.003797,0.003827,0.003595,0.002048,0.032074,0.053275,0.014932,0.006144,0.000824,0.003870,0.000366,0.000524,0.000538,0.000303


In [66]:
fig.write_image("Data/images/err_surface_strike_N_T.pdf",width = 1800, height = 700)

In [67]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2, specs=[[{'type':'surface'}, {'type':'surface'}]], subplot_titles = ['Andersen Truncated Gaussian', 'Andersen Quadratic-Exponential'])

tmp_tg_realtive_time = speed_test.pivot(index='strike', columns='N_T', values='tg_relative_speed')
fig.add_trace(go.Surface(z=tmp_tg_realtive_time.values, x=tmp_tg_realtive_time.columns, y=tmp_tg_realtive_time.index, showscale=False), row=1, col=1)

tmp_tg_realtive_time = speed_test.pivot(index='strike', columns='N_T', values='qe_relative_speed')
fig.add_trace(go.Surface(z=tmp_tg_realtive_time.values, x=tmp_tg_realtive_time.columns, y=tmp_tg_realtive_time.index, showscale=False), row=1, col=2)




max_z = max(tmp_tg_realtive_time.values.max(), tmp_tg_realtive_time.values.max())
min_z  =min(tmp_tg_realtive_time.values.min(), tmp_tg_realtive_time.values.min())

fig.update_scenes(xaxis_title_text='N_T',  
                  yaxis_title_text='Strike',  
                  zaxis_title_text='relative_speed',
                     zaxis = dict(nticks=4, range=[min_z*(1.02),max_z*(1.02)]))

fig.update_layout(title='Relative_speed',
    height=700)

fig.show()

In [69]:
fig.write_image("Data/images/relative_speed_strike_N_T.pdf",width = 1200, height = 700)

In [32]:
tmp_qe_time = speed_test.pivot(index='strike', columns='N_T', values='qe_time')
tmp_tg_time = speed_test.pivot(index='strike', columns='N_T', values='tg_time')
tmp_euler_time = speed_test.pivot(index='strike', columns='N_T', values='euler_time')

In [39]:
MC_compare_models_grid_test_asians_with_control_variate = pd.read_csv('Data/evaluation/MC_compare_models_grid_test_asians_with_control_variate.csv', index_col=0)
MC_compare_models_grid_test_asians_without_control_variate = pd.read_csv('Data/evaluation/MC_compare_models_grid_test_asians_without_control_variate.csv', index_col=0)

In [40]:
MC_compare_models_grid_test_asians_without_control_variate['scheme'].replace('simulate_heston_euler', 'simulate_heston_euler_no_cv', inplace = True)
MC_compare_models_grid_test_asians_without_control_variate['scheme'].replace('simulate_heston_andersen_tg', 'simulate_heston_andersen_tg_no_cv', inplace = True)
MC_compare_models_grid_test_asians_without_control_variate['scheme'].replace('simulate_heston_andersen_qe', 'simulate_heston_andersen_qe_no_cv', inplace = True)

In [45]:
test_cv = pd.concat([MC_compare_models_grid_test_asians_with_control_variate, MC_compare_models_grid_test_asians_without_control_variate])

In [44]:
test_cv

,scheme,heston_params#,strike,T,N_T,batch_size,absolute error,MC_price,time
0,simulate_heston_euler,3,50.0,1.0,10,10000,0.05,49.977559,0.320275
1,simulate_heston_euler,4,50.0,1.0,10,10000,0.05,50.011439,0.015201
2,simulate_heston_euler,5,50.0,1.0,10,10000,0.05,50.017955,0.027058
3,simulate_heston_euler,3,50.0,1.0,20,10000,0.05,49.966442,0.018862
4,simulate_heston_euler,4,50.0,1.0,20,10000,0.05,50.003481,0.029344
...,...,...,...,...,...,...,...,...,...
2515,simulate_heston_andersen_qe_no_cv,4,150.0,1.0,130,10000,0.05,0.011924,0.038158
2516,simulate_heston_andersen_qe_no_cv,5,150.0,1.0,130,10000,0.05,0.106615,0.037285
2517,simulate_heston_andersen_qe_no_cv,3,150.0,1.0,140,10000,0.05,0.000000,0.039488
2518,simulate_heston_andersen_qe_no_cv,4,150.0,1.0,140,10000,0.05,0.010454,0.039030


In [46]:
tmp_cv = test_cv.drop(['batch_size','absolute error'], axis=1)
speed_test_cv = pd.DataFrame()
speed_test_cv[['strike', 'N_T', 'heston_params#']] = test_cv.loc[test_cv['scheme'] =='simulate_heston_euler',['strike', 'N_T', 'heston_params#']]

speed_test_cv['qe_cv_speedup'] = tmp_cv.loc[tmp_cv['scheme'] =='simulate_heston_andersen_qe_no_cv', 'time'].reset_index(drop=True)/tmp_cv.loc[tmp_cv['scheme'] =='simulate_heston_andersen_qe', 'time'].reset_index(drop=True)

speed_test_cv['tg_cv_speedup'] = tmp_cv.loc[tmp_cv['scheme'] =='simulate_heston_andersen_tg_no_cv', 'time'].reset_index(drop=True)/tmp_cv.loc[tmp_cv['scheme'] =='simulate_heston_andersen_tg', 'time'].reset_index(drop=True)

speed_test_cv['euler_cv_speedup'] = tmp_cv.loc[tmp_cv['scheme'] =='simulate_heston_euler_no_cv', 'time'].reset_index(drop=True)/tmp_cv.loc[tmp_cv['scheme'] =='simulate_heston_euler', 'time'].reset_index(drop=True)


In [47]:
speed_test_cv

,strike,N_T,heston_params#,qe_cv_speedup,tg_cv_speedup,euler_cv_speedup
0,50.0,10,3,1.036954,1.061473,0.216748
1,50.0,10,4,3.980995,3.609496,4.446909
2,50.0,10,5,4.006464,4.057007,5.351786
3,50.0,20,3,3.944814,3.953057,5.234811
4,50.0,20,4,3.456627,3.615132,3.719907
...,...,...,...,...,...,...
835,150.0,130,4,0.856814,0.817845,0.893644
836,150.0,130,5,0.854268,0.993839,0.897874
837,150.0,140,3,0.863650,0.836486,0.890616
838,150.0,140,4,0.837527,0.829133,0.891230


In [148]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=3, specs=[[{'type':'surface'},{'type':'surface'}, {'type':'surface'}]], subplot_titles = ['Euler', 'Andersen Truncated Gaussian', 'Andersen Quadratic-Exponential'])

paramns_n = 5


euler_cv_speedup = speed_test_cv[speed_test_cv['heston_params#'] ==paramns_n ].pivot(index='strike', columns='N_T', values='euler_cv_speedup')
fig.add_trace(go.Surface(z=euler_cv_speedup.values, x=euler_cv_speedup.columns, y=euler_cv_speedup.index, showscale=False), row=1, col=1)

tg_cv_speedup = speed_test_cv[speed_test_cv['heston_params#'] ==paramns_n ].pivot(index='strike', columns='N_T', values='tg_cv_speedup')
fig.add_trace(go.Surface(z=tg_cv_speedup.values, x=tg_cv_speedup.columns, y=tg_cv_speedup.index, showscale=False), row=1, col=2)

qe_cv_speedup = speed_test_cv[speed_test_cv['heston_params#'] ==paramns_n ].pivot(index='strike', columns='N_T', values='qe_cv_speedup')
fig.add_trace(go.Surface(z=qe_cv_speedup.values, x=qe_cv_speedup.columns, y=qe_cv_speedup.index, showscale=False), row=1, col=3)

max_z = max(euler_cv_speedup.max().max(),tg_cv_speedup.values.max().max(), qe_cv_speedup.max().max())
min_z  =min(euler_cv_speedup.min().min(), tg_cv_speedup.values.min().min(), qe_cv_speedup.values.min().min())


fig.update_scenes(xaxis_title_text='N_T',  
                  yaxis_title_text='Strike',  
                  zaxis_title_text='Speed up',
                     zaxis = dict(nticks=4, range=[min_z*(1.02),max_z*(1.02)]))

fig.update_layout(#title='Control Variate Speed up Parameters #' + str(paramns_n),
    height=700)

fig.show()

In [150]:
fig.write_image("surf_" + str(paramns_n) + ".pdf",width = 1800, height = 700)

In [146]:

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=3, subplot_titles = ['Euler', 'Andersen Truncated Gaussian', 'Andersen Quadratic-Exponential'], shared_yaxes=True)

paramns_n = 5

srez  = 80

euler_cv_speedup = speed_test_cv[speed_test_cv['heston_params#'] ==paramns_n ].pivot(index='strike', columns='N_T', values='euler_cv_speedup')[srez]
fig.add_trace(go.Scatter(x= euler_cv_speedup.index , y= euler_cv_speedup.values), row=1, col=1,)

tg_cv_speedup = speed_test_cv[speed_test_cv['heston_params#'] ==paramns_n ].pivot(index='strike', columns='N_T', values='tg_cv_speedup')[srez]
fig.add_trace(go.Scatter(x= tg_cv_speedup.index , y= tg_cv_speedup.values), row=1, col=2)

qe_cv_speedup = speed_test_cv[speed_test_cv['heston_params#'] ==paramns_n ].pivot(index='strike', columns='N_T', values='qe_cv_speedup')[srez]
fig.add_trace(go.Scatter(x= qe_cv_speedup.index , y= qe_cv_speedup.values), row=1, col=3)

fig.update_layout(title='Control Variate Speed up Parameters #' + str(paramns_n)  + ' at N_T=' + str(srez),
    height=700,
    showlegend=False)


fig.update_yaxes(title_text="Speed up", row=1, col=1)
fig.update_yaxes(title_text="Speed up", row=1, col=2)
fig.update_yaxes(title_text="Speed up", row=1, col=3)
fig.update_xaxes(title_text="Strike", row=1, col=1)
fig.update_xaxes(title_text="Strike", row=1, col=2)
fig.update_xaxes(title_text="Strike", row=1, col=3)

fig.show()

In [147]:
fig.write_image("pot_" + str(paramns_n) + ".pdf",width = 1800, height = 700)

In [73]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

fig = make_subplots(rows=3, cols=3, specs=[[{'type':'surface'},{'type':'surface'}, {'type':'surface'}], [{'type':'surface'},{'type':'surface'}, {'type':'surface'}], [{'type':'surface'},{'type':'surface'}, {'type':'surface'}]], subplot_titles = ['Euler', 'Andersen Truncated Gaussian', 'Andersen Quadratic-Exponential'])

i = 1

max_z = -10000
min_z = 10000

for paramns_n in speed_test_cv['heston_params#'].unique():
    euler_cv_speedup = speed_test_cv[speed_test_cv['heston_params#'] ==paramns_n ].pivot(index='strike', columns='N_T', values='euler_cv_speedup').abs()
    fig.add_trace(go.Surface(z=euler_cv_speedup.values, x=euler_cv_speedup.columns, y=euler_cv_speedup.index, showscale=False), row=i, col=1)

    tg_cv_speedup = speed_test_cv[speed_test_cv['heston_params#'] ==paramns_n ].pivot(index='strike', columns='N_T', values='tg_cv_speedup').abs()
    fig.add_trace(go.Surface(z=tg_cv_speedup.values, x=tg_cv_speedup.columns, y=tg_cv_speedup.index, showscale=False), row=i, col=2)

    qe_cv_speedup = speed_test_cv[speed_test_cv['heston_params#'] ==paramns_n ].pivot(index='strike', columns='N_T', values='qe_cv_speedup').abs()
    fig.add_trace(go.Surface(z=qe_cv_speedup.values, x=qe_cv_speedup.columns, y=qe_cv_speedup.index, showscale=False), row=i, col=3)
    i += 1

    max_z = max(max_z, euler_cv_speedup.max().max(),tg_cv_speedup.values.max().max(), qe_cv_speedup.max().max())
    min_z  =min(max_z, euler_cv_speedup.min().min(), tg_cv_speedup.values.min().min(), qe_cv_speedup.values.min().min())


fig.update_scenes(xaxis_title_text='N_T',  
                  yaxis_title_text='Strike',  
                  zaxis_title_text='relative_speed',
                     zaxis = dict(nticks=4, range=[min_z*(1.02),max_z*(1.02)]))

fig.update_layout(title='Control Variate Speed up',
    height=2100);

#fig.show()

In [12]:
MC_compare_models_grid_test_3_andersen  = pd.read_csv('Data/evaluation/MC_compare_models_grid_test_3_andersen.csv', index_col=0)
MC_compare_models_grid_test_3_euler = pd.read_csv('Data/evaluation/MC_compare_models_grid_test_3_euler.csv', index_col=0)
MC_compare_models_grid_test_3_qe = pd.read_csv('Data/evaluation/MC_compare_models_grid_test_3_qe.csv', index_col=0)
MC_compare_models_grid_test_3_PART3 = pd.read_csv('Data/evaluation/MC_compare_models_grid_test_3_PART3.csv', index_col=0)
test = pd.concat([MC_compare_models_grid_test_3_andersen, MC_compare_models_grid_test_3_euler, MC_compare_models_grid_test_3_qe, MC_compare_models_grid_test_3_PART3])

In [6]:
test = pd.read_csv('Data\evaluation\MC_compare_models_grid_test_3_qe_tg_1.csv', index_col = 0)

In [17]:
scheme_  = 'simulate_heston_andersen_qe_cupy_old'
N_T_ = 50
heston_params_n_ = 1

df = test

test2_euler = df.loc[(df['scheme'] == scheme_) &(df['N_T'] == N_T_) &(df['heston_params#'] == heston_params_n_)]

test2_euler = test2_euler[['strike', 'T', 'MC_price', 'true']]




scheme_  = 'simulate_heston_andersen_qe_cupy_old'

test2_anderson_qe = df.loc[(df['scheme'] == scheme_) &(df['N_T'] == N_T_) &(df['heston_params#'] == heston_params_n_)]

test2_anderson_qe = test2_anderson_qe[['strike', 'T', 'MC_price', 'true']]




scheme_  = 'simulate_heston_andersen_tg_cupy_old'

test2_anderson_tg = df.loc[(df['scheme'] == scheme_) &(df['N_T'] == N_T_) &(df['heston_params#'] == heston_params_n_)]


test2_anderson_tg = test2_anderson_tg[['strike', 'T', 'MC_price', 'true']]


test2_euler['error'] = (test2_euler['MC_price'] - test2_euler['true'])#/test2_euler['true']
test2_anderson_tg['error'] = (test2_anderson_tg['MC_price'] - test2_anderson_tg['true'])#/test2_anderson_tg['true']
test2_anderson_qe['error'] = (test2_anderson_qe['MC_price'] - test2_anderson_qe['true'])#/test2_anderson_qe['true']


test2_euler = test2_euler.pivot(index='strike', columns='T', values='error')
test2_anderson_tg = test2_anderson_tg.pivot(index='strike', columns='T', values='error')
test2_anderson_qe = test2_anderson_qe.pivot(index='strike', columns='T', values='error')


In [18]:
test2_euler = test2_euler.abs()
test2_anderson_tg = test2_anderson_tg.abs()
test2_anderson_qe = test2_anderson_qe.abs()

In [19]:
test2_anderson_tg

T,0.100000,0.268966,0.437931,0.606897,0.775862,0.944828,1.113793,1.282759,1.451724,1.620690,...,3.479310,3.648276,3.817241,3.986207,4.155172,4.324138,4.493103,4.662069,4.831034,5.000000
strike,,,,,,,,,,,,,,,,,,,,,
60.000000,0.010502,0.001212,0.023585,0.031366,0.096323,0.123255,0.154780,0.222133,0.255418,0.299927,...,0.695075,0.728051,0.791129,0.806233,0.828730,0.871916,0.929052,0.943189,0.959576,1.006651
62.758621,0.027162,0.006237,0.014952,0.027817,0.095751,0.132412,0.165555,0.206003,0.237337,0.267385,...,0.691434,0.714426,0.731321,0.799153,0.801690,0.840111,0.861675,0.928437,0.950725,0.966279
65.517241,0.015934,0.002195,0.015146,0.053271,0.092195,0.101785,0.151839,0.175839,0.226931,0.280939,...,0.675591,0.703124,0.712844,0.754165,0.784523,0.817101,0.862707,0.886859,0.901321,0.934192
68.275862,0.008020,0.000647,0.018152,0.047983,0.073091,0.102306,0.143478,0.180310,0.233923,0.229687,...,0.647486,0.655376,0.690345,0.723922,0.748444,0.762635,0.832644,0.835435,0.866879,0.925597
71.034483,0.009413,0.010984,0.003387,0.016294,0.068659,0.085381,0.099286,0.157846,0.217984,0.215588,...,0.600437,0.603937,0.663901,0.690971,0.727691,0.761168,0.786824,0.800487,0.846067,0.871897
73.793103,0.020280,0.002148,0.024906,0.010672,0.047503,0.070558,0.102129,0.168035,0.178892,0.230580,...,0.559454,0.602786,0.657348,0.666039,0.667119,0.727850,0.741911,0.773698,0.806582,0.840216
76.551724,0.014828,0.016494,0.018134,0.006589,0.043281,0.052956,0.103531,0.133934,0.157452,0.194915,...,0.540288,0.565613,0.598607,0.614941,0.648876,0.660439,0.698247,0.737038,0.770987,0.790547
79.310345,0.016395,0.032957,0.011684,0.004097,0.034075,0.056881,0.081103,0.120754,0.169536,0.167034,...,0.512969,0.546078,0.550414,0.577995,0.616682,0.650918,0.670738,0.705569,0.736639,0.757610
82.068966,0.031841,0.025063,0.044665,0.007933,0.005178,0.023177,0.068241,0.103181,0.160402,0.164043,...,0.470535,0.517440,0.508331,0.560166,0.571771,0.598171,0.619267,0.652285,0.697799,0.702203


In [20]:
test2_anderson_qe.iloc[15]

T
0.100000    0.063519
0.268966    0.079437
0.437931    0.089079
0.606897    0.074521
0.775862    0.072275
0.944828    0.055240
1.113793    0.021518
1.282759    0.025691
1.451724    0.008665
1.620690    0.003435
1.789655    0.033621
1.958621    0.055621
2.127586    0.077096
2.296552    0.084643
2.465517    0.132858
2.634483    0.144107
2.803448    0.138299
2.972414    0.154516
3.141379    0.186183
3.310345    0.218368
3.479310    0.228527
3.648276    0.216410
3.817241    0.256662
3.986207    0.284803
4.155172    0.298839
4.324138    0.306252
4.493103    0.352479
4.662069    0.376137
4.831034    0.393351
5.000000    0.404950
Name: 101.3793103448276, dtype: float64

In [22]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=3, specs=[[{'type':'surface'}, {'type':'surface'}, {'type':'surface'}]], subplot_titles = ['Euler', 'Andersen Truncated Gaussian', 'Andersen Quadratic-Exponential'], shared_xaxes=True)


max_error = max(test2_euler.values.max(), test2_anderson_tg.values.max(), test2_anderson_qe.values.max())
min_error  =min(test2_euler.values.min(), test2_anderson_tg.values.min(), test2_anderson_qe.values.min())

fig.add_trace(go.Surface(z=test2_euler.values, x=test2_euler.columns, y=test2_euler.index, showscale=False), row=1, col=1, )
fig.add_trace(go.Surface(z=test2_anderson_tg.values, x=test2_anderson_tg.columns, y=test2_anderson_tg.index, showscale=False), row=1, col=2)
fig.add_trace(go.Surface(z=test2_anderson_qe.values, x=test2_anderson_qe.columns, y=test2_anderson_qe.index, showscale=False), row=1, col=3)
fig.update_scenes(xaxis_title_text='T',  
                  yaxis_title_text='Strike',  
                  zaxis_title_text='error',
                     zaxis = dict(nticks=4, range=[min_error*(1.02),max_error*(1.02)]))

fig.update_layout(
    height=700);
fig.show()

In [ ]:
fig.write_image("Data/images/err_surface_strike_T_N_T=" + str(N_T_) +"_param#" + str(heston_params_n_) + ".pdf",width = 1800, height = 700)